# Copyright

<PRE>
This notebook was created as part of the "Deep learning / VITMMA19" class at
Budapest University of Technology and Economics, Hungary,
https://portal.vik.bme.hu/kepzes/targyak/VITMMA19

Any re-use or publication of any part of the notebook is only allowed with the
 written consent of the authors.

2024 (c) Mohammed Salah Al-Radhi and Tamás Gábor Csapó (malradhi@tmit.bme.hu)
</PRE>

In [ ]:
### HYPEROPT: task during the class - we will do this together
# add WandB.ai integration to the code
# (help: https://docs.wandb.ai/guides/integrations/lightning )
# run at least 3 different trainings

In [ ]:
# install pytorch lithening
!pip install pytorch-lightning --quiet
!pip install wandb --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.8/313.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.1 MB/s eta 0:00:00


In [ ]:
import pytorch_lightning as pl
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader,random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import CIFAR10
from pytorch_lightning.loggers import WandbLogger
import wandb


In [ ]:
sweep_configuration = {
    'name': 'cifar-sweep',
    'method': 'grid',
    'parameters': {
        'learning_rate': {
            'values': [1e-3, 1e-4, 1e-5]
        },
        'optimizer': {
            'values': ['adam', 'sgd']
        },
        'neurons_FC1': {
            'values': [512, 256]
        },
        'neurons_FC2': {
            'values': [128, 64]
        },
    }
}

In [ ]:
class CifarDataModule(pl.LightningDataModule):
    def __init__(self, batch_size, data_dir="./"):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        # Define train_transform and test_transform here
        self.train_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            # Add any data augmentation transforms here, if desired
            # e.g., transforms.RandomHorizontalFlip()
        ])
        self.test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        self.num_classes = 10

    def prepare_data(self):
        CIFAR10(self.data_dir, train=True, download=True)
        CIFAR10(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            # Use self.train_transform for the training set
            cifar_full = CIFAR10(self.data_dir, train=True, transform=self.train_transform)
            self.cifar_train, self.cifar_val = random_split(cifar_full, [45000, 5000])

        if stage == 'test' or stage is None:
            # Use self.test_transform for the test set
            self.cifar_test = CIFAR10(self.data_dir, train=False, transform=self.test_transform)

    def train_dataloader(self):
        return DataLoader(self.cifar_train, batch_size=self.batch_size, shuffle=True, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.cifar_val, batch_size=self.batch_size, shuffle=False, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.cifar_test, batch_size=self.batch_size, shuffle=False, num_workers=2)

In [ ]:
class CIFAR10LitModel(pl.LightningModule):
    def __init__(self, input_shape,num_classes,learning_rate=3e-4, neurons_FC1=512, neurons_FC2=128, dropout_rate=0.5):
      super().__init__()
      self.save_hyperparameters()
      self.input_shape=input_shape
      self.learning_rate=learning_rate

      # model architecture
      self.conv1=nn.Conv2d(3,64,3,1)
      self.conv2=nn.Conv2d(64,128,3,1)
      self.conv3=nn.Conv2d(128,256,3,1)
      self.conv4=nn.Conv2d(256,512,3,1)
      self.pool1=nn.MaxPool2d(2)
      self.pool2=nn.MaxPool2d(2)

      n_sizes = self._get_output_shape(input_shape)

      self.fc1 = nn.Linear(n_sizes, neurons_FC1)  # Use neurons_FC1
      self.fc2 = nn.Linear(neurons_FC1, neurons_FC2)  # Use neurons_FC2
      self.fc3 = nn.Linear(neurons_FC2, num_classes)

      self.dropout = nn.Dropout(dropout_rate)

      self.train_acc=Accuracy(task='multiclass',num_classes=10)
      self.val_acc=Accuracy(task='multiclass',num_classes=10)
      self.test_acc=Accuracy(task='multiclass',num_classes=10)


    def _get_output_shape(self, shape):
          '''returns the size of the output tensor from the conv layers'''
          batch_size = 1
          input = torch.autograd.Variable(torch.rand(batch_size, *shape))
          output_feat = self._feature_extractor(input)
          n_size = output_feat.data.view(batch_size, -1).size(1)
          return n_size


  # conv1,relu, conv2,relu, maxpool,conv3,relu,conv4,relu,maxpool
    def _feature_extractor(self,x):
      x=F.relu(self.conv1(x))
      x=self.pool1(F.relu(self.conv2(x)))
      x=F.relu(self.conv3(x))
      x=self.pool2(F.relu(self.conv4(x)))
      return x


    def forward(self,x):
      x=self._feature_extractor(x)
      x=x.view(x.size(0),-1)
      x=F.relu(self.fc1(x))
      x = self.dropout(x)

      x=F.relu(self.fc2(x))
      x = self.dropout(x)

      x=F.log_softmax(self.fc3(x),dim=1)
      return x

    def training_step(self, batch, batch_idx):
      x, y = batch
      logits = self(x)
      loss = F.nll_loss(logits, y)
      # metric
      preds = torch.argmax(logits, dim=1)
      acc = self.train_acc(preds, y)
      self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
      self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)
      return loss

    # validation loop
    def validation_step(self, batch, batch_idx):
      x, y = batch
      logits = self(x)
      loss = F.nll_loss(logits, y)
      preds = torch.argmax(logits, dim=1)
      acc = self.val_acc(preds, y)
      self.log('val_loss', loss, prog_bar=True)
      self.log('val_acc', acc, prog_bar=True)
      return loss

    # test loop
    def test_step(self,batch,batch_idx):
      x,y=batch
      logits=self(x)
      loss=F.nll_loss(logits,y)

      pred=torch.argmax(logits,dim=1)
      acc=self.test_acc(pred,y)
      self.log('test_loss',loss,on_epoch=True)
      self.log('test_acc',acc,on_epoch=True)
      return loss

    def configure_optimizers(self):
      optimizer=torch.optim.Adam(self.parameters(),self.learning_rate)
      return optimizer


In [ ]:
# class for visualizing one batch of validation images along with predicted and rall class label
class ImagePredictionLogger(pl.Callback):
    def __init__(self, val_samples, num_samples=32):
        super().__init__()
        self.val_imgs, self.val_labels = val_samples
        self.val_imgs = self.val_imgs[:num_samples]
        self.val_labels = self.val_labels[:num_samples]

    def on_validation_epoch_end(self, trainer, pl_module):
        val_imgs = self.val_imgs.to(device=pl_module.device)
        logits = pl_module(val_imgs)
        preds = torch.argmax(logits, 1)

        trainer.logger.experiment.log({
            "examples": [wandb.Image(x, caption=f"Pred:{pred}, Label:{y}")
                            for x, pred, y in zip(val_imgs, preds, self.val_labels)],
            "global_step": trainer.global_step
            })

In [ ]:
# Instantiate the cifar and model
cifar = CifarDataModule(batch_size=32)
cifar.prepare_data()
cifar.setup()

# Grab samples to log predictions on
samples = next(iter(cifar.val_dataloader()))

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def train_model(learning_rate=1e-3, optimizer='adam', neurons_FC1=512, neurons_FC2=128):
    wandb.login(key='')
    config = wandb.config
    wandb_logger = WandbLogger(project='lastt', job_type='train', log_model="all")

    # Instantiate classes
    dm = CifarDataModule(32)
    dm.prepare_data()
    dm.setup()

    # Update model initialization to use the new parameters
    model = CIFAR10LitModel((3, 32, 32), dm.num_classes, learning_rate, neurons_FC1, neurons_FC2)
    wandb_logger.watch(model)

    # Initialize Callbacks
    checkpoint_callback = pl.callbacks.ModelCheckpoint()
    early_stop_callback = pl.callbacks.EarlyStopping(monitor="val_acc", patience=3, verbose=False, mode="max")

    # Trainer
    trainer = pl.Trainer(max_epochs=10,
                     logger=wandb_logger,
                     callbacks=[checkpoint_callback, early_stop_callback, ImagePredictionLogger(samples)]
                    )
    # Train the model
    trainer.fit(model, dm)

    # Evaluate the model
    trainer.test(dataloaders=dm.test_dataloader())
    # Tell WandB you have finished
    wandb.finish()


In [ ]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project='lastt')

Create sweep with ID: 5xpd0uvu
Sweep URL: https://wandb.ai/ayushshende10-budapesti-m-szaki-s-gazdas-gtudom-nyi-egyetem/lastt/sweeps/5xpd0uvu


In [ ]:
wandb.agent(sweep_id, function=train_model, count=10)

wandb: Agent Starting Run: utv6vgt1 with config:
wandb: 	learning_rate: 0.001
wandb: 	neurons_FC1: 512
wandb: 	neurons_FC2: 128
wandb: 	optimizer: adam


Files already downloaded and verified
Files already downloaded and verified


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


...

Files already downloaded and verified
Files already downloaded and verified


INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at ./lastt/5tnatk9t/checkpoints/epoch=9-step=14070.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'neurons_FC1' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'neurons_FC2' was locked by 'sweep' (ignored update).
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at ./lastt/5tnatk9t/checkpoints/epoch=9-step=14070.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7508999705314636     │
│         test_loss         │    0.8460046648979187     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇███
global_step,▁▂▂▃▄▅▅▆▇▇█
test_acc,▁
test_loss,▁
train_acc_epoch,▁▄▅▆▆▇▇███
train_acc_step,▁▁▂▃▂▆▃▄▃▆▄▅▆▅▅▅▇▇▆▅▅▆▆▅▇█▇▇█▆▇▆▇█▇▇▇██▇
train_loss_epoch,█▅▄▃▃▂▂▂▁▁
train_loss_step,█▅▆▇▆▄▄▇▄▅▄▄▃▇▇▆▃▁▅▅▃▂▅▄▅▅▄▃▃▃▅▇▂▂▂▄▁▂▄▃
trainer/global_step,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇████████
val_acc,▁▅▆▇▇▇▇███
val_loss,█▄▂▂▁▁▂▁▁▂


In [ ]:
### TASK OF THE STUDENT

# extend WandB.ai integration in the code with sweeps
# (e.g. add variables like learning rate, optimizer, neurons_FC1, neurons_FC2)
# help: https://docs.wandb.ai/guides/sweeps and
#       https://github.com/wandb/wandb/issues/5003
# store the hyperparameters and val_acc to wandb
# run at least 10 trainings
# in wandb.ai, export the result of the runs as a .csv file,
# in wandb.ai, create a report from the sweep results and share it by submitting
# the link in Moodle.